# Installlations

In [ ]:
!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  
!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}+${CUDA}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}+${CUDA}.html
!pip install torch-geometric
#!pip install -U sentence-transformers

Found existing installation: torch-scatter 2.0.9
Uninstalling torch-scatter-2.0.9:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/torch_scatter-2.0.9.dist-info/*
    /usr/local/lib/python3.7/dist-packages/torch_scatter/*
Proceed (y/n)?   ERROR: Operation cancelled by user
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-+.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-+.html
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During 

In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Model definitions

In [ ]:
import torch
from torch_geometric.datasets import UPFD
import numpy as np
from torch_geometric.loader import DataLoader

In [ ]:
import torch.nn.functional as F
from torch.nn import ReLU, LeakyReLU, Softmax, Linear, SELU, GELU, ELU, Dropout
from torch_geometric.nn import SAGEConv, global_max_pool, TopKPooling

In [ ]:
from torch.autograd import Variable
from sklearn.metrics import accuracy_score, f1_score 

In [ ]:
test_data_pol = UPFD(root=".", name="politifact", feature="bert",split="test")
train_data_pol = UPFD(root=".", name="politifact", feature="bert", split="train")
val_data_pol = UPFD(root=".", name="politifact", feature="bert", split="val")

In [ ]:
# combined = test_data_pol+train_data_pol+val_data_pol

# train_data_pol,val_data_pol,test_data_pol = torch.utils.data.random_split(combined, [220,31,63],generator=torch.Generator().manual_seed(42))

In [ ]:
train_data_pol

UPFD(62, name=politifact, feature=bert)

In [ ]:
test_loader = DataLoader(test_data_pol, batch_size=32, shuffle=True)
train_loader = DataLoader(train_data_pol, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data_pol, batch_size=32, shuffle=True)

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(Net, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels[0])
        self.leaky1 = LeakyReLU()
        self.conv2 = SAGEConv(hidden_channels[0], hidden_channels[1])
        self.leaky2 = LeakyReLU()
        self.conv3 = SAGEConv(hidden_channels[1], hidden_channels[2])
        self.leaky3 = LeakyReLU() 
        
        self.full1 = Linear(hidden_channels[2],hidden_channels[3])
        self.ge1 = GELU()
        self.full2 = Linear(hidden_channels[3],hidden_channels[4])
        self.ge2 = GELU()
        self.full3 = Linear(hidden_channels[4],out_channels)


    def forward(self, x, edge_index, batch):
        h = self.leaky1(self.conv1(x, edge_index))
        h = self.leaky2(self.conv2(h, edge_index))
        h = self.leaky3(self.conv3(h, edge_index))

        h = global_max_pool(h,batch)

        h = self.ge1(self.full1(h))
        h = self.ge2(self.full2(h))
        h = self.full3(h)

        return torch.sigmoid(h)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper paarmeter tuning

In [ ]:
def objective(trial):
  
  model = Net(768,[trial.suggest_int(name="layer_size1",low=256,high=512,step=128),
                   trial.suggest_int(name="layer_size2",low=256,high=512,step=128),
                   trial.suggest_int(name="layer_size3",low=256,high=512,step=128),
                   trial.suggest_int(name="layer_size4",low=64,high=256,step=64),
                   trial.suggest_int(name="layer_size5",low=64,high=256,step=64),
  
  ],1).to(device)
  optimizer = torch.optim.Adam(model.parameters(),lr=trial.suggest_loguniform('learning_rate', 1e-6, 1e-3),betas=(trial.suggest_loguniform('b1', 1-1e-1,1-1e-3),0.99))
  lossff = torch.nn.BCELoss()

  total_loss = 0
  weighted_loss = 0
  exp_param = 0.8

  wloss = []
  best_val = 1

  for i in range(1600):
    print("Epoch:", i)
    model.train()
    for data in train_loader:
          data = data.to(device)
          optimizer.zero_grad()
          out = model(data.x, data.edge_index, data.batch)
          loss = lossff(torch.reshape(out,(-1,)), data.y.float())
          loss.backward()
          optimizer.step()
          total_loss += float(loss) * data.num_graphs
    print("Train: ",total_loss / len(train_loader.dataset))

    model.eval()
    total_loss = 0
    for data in val_loader:
          data = data.to(device)
          out = model(data.x, data.edge_index, data.batch)
          loss = lossff(torch.reshape(out,(-1,)), data.y.float())
          total_loss += float(loss) * data.num_graphs
          
    print("Test", total_loss / len(val_loader.dataset))

    weighted_loss = exp_param*(weighted_loss) + (1-exp_param)*(total_loss/ len(val_loader.dataset))
    print(weighted_loss/(1-exp_param**(i+1)))
    wloss.append(weighted_loss/(1-exp_param**(i+1)))

    if(best_val>weighted_loss/(1-exp_param**(i+1))):
      best_val = weighted_loss/(1-exp_param**(i+1))

    if(i-10>=0 and wloss[i-10]-weighted_loss<-0.01):
      break

  return best_val

In [ ]:
import optuna
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=20)

[I 2022-10-16 08:16:04,478] A new study created in memory with name: no-name-1e8ef305-5212-47c4-a7fd-4ef3306f7435
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  # Remove the CWD from sys.path while we load stuff.


Epoch: 0
Train:  0.6879171094586772
Test 0.699082612991333
0.699082612991333
Epoch: 1
Train:  1.0370765482225726
Test 0.6996992826461792
0.6994252072440253
Epoch: 2
Train:  1.03695761388348
Test 0.700280487537384
0.6997757319544183
Epoch: 3
Train:  1.0366437098672312
Test 0.7006199955940247
0.7000617291223066
Epoch: 4
Train:  1.0365300899551761
Test 0.7010000348091125
0.700340853850652
Epoch: 5
Train:  1.0366294220570595
Test 0.7014452815055847
0.7006402151501285
Epoch: 6
Train:  1.0363946374385589
Test 0.7018416523933411
0.7009442668649565
Epoch: 7
Train:  1.036256968013702
Test 0.7022421360015869
0.7012561692249233
Epoch: 8
Train:  1.0362733775569546
Test 0.7026827931404114
0.7015857264269187
Epoch: 9
Train:  1.036109584954477
Test 0.7031116485595703
0.701927621564291
Epoch: 10
Train:  1.035969365027643
Test 0.7035372257232666
0.7022797937570996
Epoch: 11
Train:  1.0359365747820946
Test 0.7040051817893982
0.7026503347464571
Epoch: 12
Train:  1.0359328783327533
Test 0.7044985890388489

[I 2022-10-16 08:16:12,161] Trial 0 finished with value: 0.5168222606771309 and parameters: {'layer_size1': 384, 'layer_size2': 256, 'layer_size3': 512, 'layer_size4': 256, 'layer_size5': 128, 'learning_rate': 1.7269151676184985e-05, 'b1': 0.9494564658986889}. Best is trial 0 with value: 0.5168222606771309.


Train:  0.3164188884679348
Test 0.5362651944160461
0.5245158449125862
Epoch: 166
Train:  0.31888307823288825
Test 0.5446001887321472
0.5285327136764985
Epoch: 167
Train:  0.32077544795409324
Test 0.5491675138473511
0.5326596737106689
Epoch: 0
Train:  0.6874574403608998
Test 0.7044631838798523
0.7044631838798522
Epoch: 1
Train:  1.0372989129635595
Test 0.7082279324531555
0.7065547108650208
Epoch: 2
Train:  1.0367923965377193
Test 0.712127685546875
0.7088387168821741
Epoch: 3
Train:  1.0368902241030047
Test 0.7167356610298157
0.7115138334633535
Epoch: 4
Train:  1.0375670627240212
Test 0.7220823764801025
0.7146577360742379
Epoch: 5
Train:  1.0389100197822816
Test 0.7286458015441895
0.7184492795379902
Epoch: 6
Train:  1.0401950324735334
Test 0.7325580716133118
0.7220198384239315
Epoch: 7
Train:  1.0410758074252837
Test 0.7357912063598633
0.7253293571095673
Epoch: 8
Train:  1.0421648871514104
Test 0.7399211525917053
0.7287001342562118
Epoch: 9
Train:  1.0425259172916412
Test 0.7425117492675

[I 2022-10-16 08:16:14,395] Trial 1 finished with value: 0.5954291380651352 and parameters: {'layer_size1': 256, 'layer_size2': 512, 'layer_size3': 384, 'layer_size4': 192, 'layer_size5': 256, 'learning_rate': 9.006247606734518e-05, 'b1': 0.9747253956189821}. Best is trial 0 with value: 0.5168222606771309.


...

Train:  0.8135142980083343
Test 0.6607689261436462
0.6614920473926655
Epoch: 1596
Train:  0.8130209849726769
Test 0.6605861186981201
0.6613108616537564
Epoch: 1597
Train:  0.8125231804386261
Test 0.6604032516479492
0.661129339652595
Epoch: 1598
Train:  0.8120240492205466
Test 0.6602204442024231
0.6609475605625607
Epoch: 1599
Train:  0.8115128192209429
Test 0.6600378155708313
0.6607656115642149
Epoch: 0
Train:  0.6980514834004063
Test 0.6900963187217712
0.6900963187217712
Epoch: 1
Train:  1.0412802282840974
Test 0.6914088129997253
0.6908254822095237
Epoch: 2
Train:  1.040057219805256
Test 0.6927382946014404
0.6916094217144075
Epoch: 3
Train:  1.0389869713014173
Test 0.694089412689209
0.6924495270581749
Epoch: 4
Train:  1.038532558948763
Test 0.6956215500831604
0.6933931321941178
Epoch: 5
Train:  1.037752373564628
Test 0.6972915530204773
0.6944498209860662
Epoch: 6
Train:  1.0371971409167013
Test 0.6991074681282043
0.6956285472230224
Epoch: 7
Train:  1.0362956052826298
Test 0.70110911130

[I 2022-10-16 08:25:47,047] Trial 18 finished with value: 0.6900963187217712 and parameters: {'layer_size1': 384, 'layer_size2': 384, 'layer_size3': 256, 'layer_size4': 128, 'layer_size5': 64, 'learning_rate': 5.594360103803041e-05, 'b1': 0.9609196928489294}. Best is trial 11 with value: 0.46340372127499985.


Test 0.7234838008880615
0.712114223995109
Epoch: 15
Train:  1.0361745395968038
Test 0.7279590368270874
0.7153749683558033
Epoch: 16
Train:  1.0368868535564792
Test 0.7324461936950684
0.7188678664975033
Epoch: 17
Train:  1.038871628622855
Test 0.737126350402832
0.7225865531824436
Epoch: 0
Train:  0.6846435858357337
Test 0.7103950381278992
0.7103950381278991
Epoch: 1
Train:  1.0373644184681676
Test 0.717557966709137
0.7143744428952536
Epoch: 2
Train:  1.038367530030589
Test 0.7242490649223328
0.7184214191358599
Epoch: 3
Train:  1.0396320733331865
Test 0.7302464246749878
0.7224271798198463
Epoch: 4
Train:  1.0397402663384714
Test 0.7314172387123108
0.72510151909057
Epoch: 5
Train:  1.0376452563270446
Test 0.7328222990036011
0.7271942797140606
Epoch: 6
Train:  1.0353121719052714
Test 0.7362664341926575
0.7294901999895351
Epoch: 7
Train:  1.0333417798242261
Test 0.7333030700683594
0.7304065043705142
Epoch: 8
Train:  1.0267785768355093
Test 0.735419750213623
0.7315645890311131
Epoch: 9
Train

[I 2022-10-16 08:25:48,704] Trial 19 finished with value: 0.5618630652914107 and parameters: {'layer_size1': 512, 'layer_size2': 512, 'layer_size3': 384, 'layer_size4': 64, 'layer_size5': 128, 'learning_rate': 0.0001486254774652934, 'b1': 0.9212754626534484}. Best is trial 11 with value: 0.46340372127499985.


Train:  0.36950270413991904
Test 0.740239143371582
0.6356718988079805


In [ ]:
best_trial = study.best_trial

In [ ]:
print(best_trial.params)

{'layer_size1': 512, 'layer_size2': 512, 'layer_size3': 256, 'layer_size4': 64, 'layer_size5': 64, 'learning_rate': 4.531486810010387e-06, 'b1': 0.9591472429391925}


In [ ]:
import json

In [ ]:
with open('mydata.json', 'w') as f:
    json.dump(best_trial.params, f)

# Best Model training

In [ ]:
model = Net(768,[512,512,256,64,64],1).to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=4.531486810010387e-06,betas=(0.9591472429391925,0.99))
lossff = torch.nn.BCELoss()

total_loss = 0
weighted_loss = 0
exp_param = 0.8

wloss = []
best_val = 1

for i in range(2500):
  print("Epoch:", i)
  model.train()
  for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
  print("Train: ",total_loss / len(train_loader.dataset))

  model.eval()
  total_loss = 0
  for data in val_loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        total_loss += float(loss) * data.num_graphs
        
  print("Test", total_loss / len(val_loader.dataset))

  weighted_loss = exp_param*(weighted_loss) + (1-exp_param)*(total_loss/ len(val_loader.dataset))
  print(weighted_loss/(1-exp_param**(i+1)))
  wloss.append(weighted_loss/(1-exp_param**(i+1)))

  if(best_val>weighted_loss/(1-exp_param**(i+1))):
    best_val = weighted_loss/(1-exp_param**(i+1))

  if(best_val<0.50):
    break

Epoch: 0
Train:  0.6956658055705409
Test 0.6910093426704407
0.6910093426704407
Epoch: 1
Train:  1.0410125284425673
Test 0.6911178231239319
0.6910696095890471
Epoch: 2
Train:  1.0409150421619415
Test 0.6912251114845276
0.6911333398740801
Epoch: 3
Train:  1.0407743944275765
Test 0.6913238763809204
0.691197884761221
Epoch: 4
Train:  1.0406961921722657
Test 0.6914269924163818
0.6912660391089008
Epoch: 5
Train:  1.0405746679152212
Test 0.6915289163589478
0.6913372934593561
Epoch: 6
Train:  1.0404889564360342
Test 0.6916330456733704
0.6914121404547721
Epoch: 7
Train:  1.0403911384844011
Test 0.6917358636856079
0.6914899372347586
Epoch: 8
Train:  1.0403199964954006
Test 0.691839337348938
0.6915706503945169
Epoch: 9
Train:  1.0401819655972142
Test 0.6919368505477905
0.6916527004867399
Epoch: 10
Train:  1.0401239741233088
Test 0.6920366287231445
0.6917367017911238
Epoch: 11
Train:  1.0400205677555454
Test 0.6921361684799194
0.691822490482891
Epoch: 12
Train:  1.0399190668136842
Test 0.692236781

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
@torch.no_grad()
def val(model):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    for data in val_loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        # print(out)
        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        # print(loss)
        total_loss += float(loss) * data.num_graphs
        all_preds.append(torch.reshape(out, (-1,)))
        all_labels.append(data.y.float())
    # print(all_preds)
    accuracy, f1 = metrics(all_preds, all_labels)
    return total_loss / len(val_loader.dataset), accuracy, f1

@torch.no_grad()
def test(model):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    for data in test_loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        # print(out)
        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        # print(loss)
        total_loss += float(loss) * data.num_graphs
        all_preds.append(torch.reshape(out, (-1,)))
        all_labels.append(data.y.float())
    # print(all_preds)
    accuracy, f1 = metrics(all_preds, all_labels)
    return total_loss / len(test_loader.dataset), accuracy, f1


def metrics(preds, gts):
    preds = torch.round(torch.cat(preds))
    gts = torch.cat(gts)
    acc = accuracy_score(preds.cpu().numpy(), gts.cpu().numpy())
    f1 = f1_score(preds.cpu().numpy(), gts.cpu().numpy())
    return acc, f1

In [ ]:
val(model),test(model)

((0.4973466098308563, 0.8064516129032258, 0.8125000000000001),
 (0.47412919175570906, 0.8009049773755657, 0.7864077669902911))

In [ ]:
final = DataLoader(train_data_pol+val_data_pol, batch_size=32, shuffle=True)

In [ ]:
model = Net(768,[512,512,256,64,64],1).to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=4.531486810010387e-06,betas=(0.9591472429391925,0.99))
lossff = torch.nn.BCELoss()

total_loss = 0
weighted_loss = 0
exp_param = 0.8

wloss = []

for i in range(1360):
  print("Epoch:", i)
  model.train()
  for data in final:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
  print("Train: ",total_loss / len(final.dataset))

  model.eval()
  total_loss = 0
  for data in test_loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        loss = lossff(torch.reshape(out,(-1,)), data.y.float())
        total_loss += float(loss) * data.num_graphs
        
  print("Test", total_loss / len(test_loader.dataset))

  weighted_loss = exp_param*(weighted_loss) + (1-exp_param)*(total_loss/ len(test_loader.dataset))
  print(weighted_loss/(1-exp_param**(i+1)))
  wloss.append(weighted_loss/(1-exp_param**(i+1)))

  # if(i-30>=0 and wloss[i-20]-weighted_loss<0.001):
  #   break

print(weighted_loss)

Epoch: 0
Train:  0.6932850473670549
Test 0.6930371671240794
0.6930371671240794
Epoch: 1
Train:  2.340089228845412
Test 0.6930178231243634
0.6930264204575707
Epoch: 2
Train:  2.3399687018445743
Test 0.6930029516306399
0.6930168020858777
Epoch: 3
Train:  2.3398405185309787
Test 0.692983571490551
0.6930055451091409
Epoch: 4
Train:  2.339719251278908
Test 0.6929662049625793
0.692993842304952
Epoch: 5
Train:  2.339601752578571
Test 0.6929479478710917
0.6929814023616947
Epoch: 6
Train:  2.3394957357837307
Test 0.6929307356139653
0.6929685799595103
Epoch: 7
Train:  2.33940339280713
Test 0.6929170195333558
0.6929561890195113
Epoch: 8
Train:  2.3393016822876467
Test 0.6929018354523775
0.6929436330758539
Epoch: 9
Train:  2.3391974741412747
Test 0.6928860453458933
0.6929307300813796
Epoch: 10
Train:  2.339099100840989
Test 0.6928698989600618
0.6929174205796205
Epoch: 11
Train:  2.339007043710319
Test 0.6928548327398516
0.6929039793364706
Epoch: 12
Train:  2.3389083050912425
Test 0.692839753304132

KeyboardInterrupt: ignored

In [ ]:
test(model)

(0.4395631611077494, 0.8552036199095022, 0.851851851851852)

In [ ]:
torch.save(model.state_dict(), "final.pt")

### Best Metrics found after training 
**Epoch: 114 |  TestLoss: 0.4395631611077494| TestAcc: 0.8552036199095022 | TestF1: 0.851851851851852**

*Note : We choose the best epoch based on lowest Validation Loss.*
